
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
 !pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 8.25 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y4", label="solidos")

/tmp/ipykernel_12269/178310800.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-29 16:33:18.261320: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 16:33:20.500917: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 16:33:20.501009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 16:33:20.837922: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 16:33:21.559478: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 16:33:21.564604: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [11]:
tester.Loop(sort_by='r2',
            boundarie=0.5,
            hidden_sizes = [[20,15], [19,17], [16, 15]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02])

Testando combinacao1: Hidden Size=[20, 15], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++


Elapsed time:  2.214763517999927
Stopped at epoch:  62
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.22570555199945375
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.2452242869994734
Stopped at epoch:  7
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.20994523800072784
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.2867715649999809
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.22609308699975372
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  0.4468213039999682
Stopped at epoch:  13
1/1 [==============================] - 0s 19ms/step
+

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_5_1_0,0.521442,0.704007,0.465651,0.577811,57.714226,23.210066,61.476967,41.217979
model_5_1_1,0.582950,0.686309,0.458607,0.566379,50.296398,24.597822,62.287376,42.334030
model_5_1_2,0.586258,0.617440,0.383751,0.495583,49.897434,29.998186,70.899513,49.245762
model_5_1_3,0.633488,0.651119,0.223336,0.420943,44.201485,27.357204,89.355324,56.532860
model_5_1_23,0.679168,0.212533,-0.483607,-0.157593,38.692451,61.748665,170.689209,113.014809
model_5_1_4,0.679731,0.713314,0.182400,0.424689,38.624550,22.480236,94.065002,56.167084
model_5_1_24,0.683712,0.240184,-0.481032,-0.144407,38.144375,59.580391,170.392853,111.727463
model_5_1_22,0.693562,0.275965,-0.405551,-0.087335,36.956455,56.774704,161.708832,106.155579
model_5_1_21,0.697624,0.382701,-0.444441,-0.063514,36.466686,48.405010,166.183121,103.830009
model_5_1_20,0.697789,0.369371,-0.412466,-0.051451,36.446789,49.450291,162.504364,102.652229


DataFrame salvo em ../results/metrics_5_1
+++++++++++ [5_2] | 1 ++++++++++++++++++
Elapsed time:  0.24952939300055732
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [5_2] | 2 ++++++++++++++++++
Elapsed time:  0.23774320599932253
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [5_2] | 3 ++++++++++++++++++
Elapsed time:  0.231946398999753
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [5_2] | 4 ++++++++++++++++++
Elapsed time:  0.21702379399994243
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [5_2] | 5 ++++++++++++++++++
Elapsed time:  0.2341462340000362
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [5_2] | 6 ++++++++++++++++++
Elapsed time:  0.2202951409999514
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [5_2] | 7 ++++++++++++++++++
Elapsed time:  0.2168502720

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_5_2_1,0.684573,0.863157,-0.805575,0.484430,38.040565,29.105965,131.603119,77.339920
model_5_2_2,0.684745,0.863799,-0.843566,0.476225,38.019817,28.969503,134.372162,78.570740
model_5_2_0,0.686260,0.854337,-0.742133,0.492316,37.837120,30.981865,126.979034,76.157021
model_5_2_24,0.691895,0.814595,-1.445875,0.301572,37.157520,39.434895,178.272781,104.770226
model_5_2_23,0.692159,0.815125,-1.439395,0.303452,37.125675,39.322098,177.800430,104.488235
model_5_2_4,0.693143,0.818324,-0.950914,0.417545,37.007099,38.641727,142.196472,87.373352
model_5_2_3,0.698914,0.876552,-0.883891,0.476576,36.311108,26.256830,137.311386,78.518181
model_5_2_22,0.699691,0.831548,-1.403896,0.323895,36.217323,35.829170,175.213013,101.421593
model_5_2_21,0.705130,0.844372,-1.380839,0.338794,35.561440,33.101566,173.532455,99.186691
model_5_2_19,0.707522,0.846974,-1.348720,0.348092,35.272957,32.547958,171.191391,97.791901


...

DataFrame salvo em ../results/metrics_5_7
+++++++++++ [5_8] | 1 ++++++++++++++++++
Elapsed time:  0.22574269800043112
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_8] | 2 ++++++++++++++++++
Elapsed time:  0.23894462499993097
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_8] | 3 ++++++++++++++++++
Elapsed time:  0.23585574600019754
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_8] | 4 ++++++++++++++++++
Elapsed time:  0.22355904200048826
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_8] | 5 ++++++++++++++++++
Elapsed time:  0.4176643780001541
Stopped at epoch:  13
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_8] | 6 ++++++++++++++++++
Elapsed time:  0.21349886299958598
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_8] | 7 ++++++++++++++++++
Elapsed time:  1.120575

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_5_8_24,0.840576,0.254268,0.632784,0.454291,19.226572,73.976044,40.682606,58.308495
model_5_8_23,0.841196,0.254618,0.637202,0.456618,19.151846,73.941292,40.193184,58.059822
model_5_8_3,0.842040,0.419248,0.849855,0.641295,19.050022,57.610161,16.634098,38.327332
model_5_8_22,0.843401,0.259525,0.651426,0.465970,18.885878,73.454605,38.617367,57.060555
model_5_8_21,0.844600,0.263050,0.659320,0.471555,18.741228,73.104919,37.742817,56.463833
model_5_8_20,0.845064,0.265984,0.660547,0.473595,18.685278,72.813812,37.606869,56.245853
model_5_8_4,0.848593,0.366416,0.846699,0.613787,18.259686,62.851048,16.983662,41.266445
model_5_8_5,0.849343,0.372074,0.839542,0.613076,18.169315,62.289822,17.776623,41.342419
model_5_8_19,0.850673,0.269970,0.701997,0.495780,18.008823,72.418404,33.014717,53.875458
model_5_8_18,0.851831,0.273034,0.710889,0.501623,17.869184,72.114456,32.029625,53.251076


DataFrame salvo em ../results/metrics_5_8
+++++++++++ [5_9] | 1 ++++++++++++++++++
Elapsed time:  0.20570939499975793
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_9] | 2 ++++++++++++++++++
Elapsed time:  0.28373717199974635
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_9] | 3 ++++++++++++++++++
Elapsed time:  0.353117882999868
Stopped at epoch:  11
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_9] | 4 ++++++++++++++++++
Elapsed time:  0.5049726139995983
Stopped at epoch:  14
1/1 [==============================] - 0s 15ms/step
+++++++++++ [5_9] | 5 ++++++++++++++++++
Elapsed time:  0.39178493400049774
Stopped at epoch:  12
1/1 [==============================] - 0s 13ms/step
+++++++++++ [5_9] | 6 ++++++++++++++++++
Elapsed time:  0.43572089899953426
Stopped at epoch:  13
1/1 [==============================] - 0s 13ms/step
+++++++++++ [5_9] | 7 ++++++++++++++++++
Elapsed time:  0.26654

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_5_9_24,0.835006,-0.425999,0.948959,0.597267,19.898373,104.042488,9.324321,59.469170
model_5_9_23,0.835156,-0.423513,0.948610,0.597714,19.880213,103.861084,9.387999,59.403095
model_5_9_22,0.835451,-0.419686,0.948382,0.598582,19.844711,103.581856,9.429643,59.274956
model_5_9_21,0.835999,-0.413406,0.948231,0.600137,19.778612,103.123680,9.457306,59.045372
model_5_9_20,0.836798,-0.407078,0.949118,0.602308,19.682211,102.661980,9.295226,58.724709
model_5_9_19,0.837541,-0.398945,0.948989,0.604361,19.592587,102.068535,9.318812,58.421539
model_5_9_18,0.838491,-0.389576,0.949439,0.607073,19.478079,101.384979,9.236535,58.021053
model_5_9_17,0.839884,-0.375991,0.950004,0.610956,19.310009,100.393814,9.133379,57.447689
model_5_9_16,0.840033,-0.375454,0.950471,0.611369,19.292057,100.354652,9.048130,57.386765
model_5_9_15,0.840057,-0.375956,0.950814,0.611438,19.289209,100.391304,8.985343,57.376617


DataFrame salvo em ../results/metrics_5_9
Testando combinacao6: Hidden Size=[16, 15], regularizer=0.1, learning_rate=0.02
+++++++++++ [6_0] | 1 ++++++++++++++++++
Elapsed time:  0.4139069709999603
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [6_0] | 2 ++++++++++++++++++
Elapsed time:  0.02540389400019194
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [6_0] | 3 ++++++++++++++++++
Elapsed time:  0.026205326999843237
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [6_0] | 4 ++++++++++++++++++
Elapsed time:  0.0260196300005191
Stopped at epoch:  0
1/1 [==============================] - 0s 13ms/step
+++++++++++ [6_0] | 5 ++++++++++++++++++
Elapsed time:  0.026258865000272635
Stopped at epoch:  0
1/1 [==============================] - 0s 13ms/step
+++++++++++ [6_0] | 6 ++++++++++++++++++
Elapsed time:  0.025807932000134315
Stopped at epoch:  0
1/1 [==============================